In [5]:
from typing import Dict
import numpy as np
import ray
import os

In [6]:
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [11]:
ds = ray.data.read_tfrecords(["/mnt/750G/data/droid/1.0.0/"])

In [8]:
ds

2024-12-20 19:37:26,453	INFO streaming_executor.py:108 -- Starting execution of Dataset. Full logs are in /tmp/ray/session_2024-12-20_19-35-50_515808_8991/logs/ray-data
2024-12-20 19:37:26,453	INFO streaming_executor.py:109 -- Execution plan of Dataset: InputDataBuffer[Input] -> TaskPoolMapOperator[ReadTFRecord]


Running 0: 0.00 row [00:00, ? row/s]

- ReadTFRecord->SplitBlocks(2) 1: 0.00 row [00:00, ? row/s]

(ReadTFRecord->SplitBlocks(2) pid=9546) 2024-12-20 19:37:28.723850: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(ReadTFRecord->SplitBlocks(2) pid=9546) WARNING: All log messages before absl::InitializeLog() is called are written to STDERR
(ReadTFRecord->SplitBlocks(2) pid=9546) E0000 00:00:1734712648.924386    9546 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
(ReadTFRecord->SplitBlocks(2) pid=9546) E0000 00:00:1734712648.986835    9546 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
(ReadTFRecord->SplitBlocks(2) pid=9546) 2024-12-20 19:37:29.466026: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions

KeyboardInterrupt: 

In [12]:
data = ds.take(21)

2024-12-20 19:56:34,467	INFO streaming_executor.py:108 -- Starting execution of Dataset. Full logs are in /tmp/ray/session_2024-12-20_19-35-50_515808_8991/logs/ray-data
2024-12-20 19:56:34,490	INFO streaming_executor.py:109 -- Execution plan of Dataset: InputDataBuffer[Input] -> TaskPoolMapOperator[ReadTFRecord] -> LimitOperator[limit=21]


Running 0: 0.00 row [00:00, ? row/s]

- ReadTFRecord->SplitBlocks(2) 1: 0.00 row [00:00, ? row/s]

- limit=21 2: 0.00 row [00:00, ? row/s]

In [13]:
import io
from PIL import Image

In [19]:
len(data[1]['steps/observation/exterior_image_1_left'])

477

In [20]:
for i in range(len(data)):
    for u in range(len(data[i]['steps/observation/exterior_image_1_left'])):
        img = Image.open(io.BytesIO(data[i]['steps/observation/exterior_image_1_left'][u]))
        tmp = img.save(f"/mnt/750G/data/droid_left/{i+17}_{u}.png")

In [49]:
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import io

In [ ]:
i[0]

In [ ]:
plt.imshow(Image.open(io.BytesIO(i[1])))

In [2]:
ds = ray.data.read_images("/mnt/750G/data/droid_all")

2024-12-14 16:52:16,008	INFO worker.py:1821 -- Started a local Ray instance.


In [3]:
ds

2024-12-14 16:52:40,655	INFO streaming_executor.py:108 -- Starting execution of Dataset. Full logs are in /tmp/ray/session_2024-12-14_16-52-13_202376_372677/logs/ray-data
2024-12-14 16:52:40,655	INFO streaming_executor.py:109 -- Execution plan of Dataset: InputDataBuffer[Input] -> TaskPoolMapOperator[ReadImage]


Running 0: 0.00 row [00:00, ? row/s]

- ReadImage 1: 0.00 row [00:00, ? row/s]

2024-12-14 16:52:51,741	INFO streaming_executor.py:108 -- Starting execution of Dataset. Full logs are in /tmp/ray/session_2024-12-14_16-52-13_202376_372677/logs/ray-data
2024-12-14 16:52:51,743	INFO streaming_executor.py:109 -- Execution plan of Dataset: InputDataBuffer[Input] -> TaskPoolMapOperator[ReadImage]


Running 0: 0.00 row [00:00, ? row/s]

- ReadImage 1: 0.00 row [00:00, ? row/s]

Dataset(
   num_rows=22056,
   schema={image: numpy.ndarray(shape=(180, 320, 3), dtype=uint8)}
)

: 

In [4]:
ray.shutdown()